In [5]:
!pip install transformers -q 
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu -q 

In [6]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

model_name = "lewisnjue/my_awesome_food_model"

# Download the model and processor
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)


/home/gitpod/.pyenv/versions/3.12.9/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [8]:
import torch
from PIL import Image

import requests

In [9]:
image = Image.open(requests.get("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/coco_sample.png", stream=True).raw)

In [10]:
inputs = processor(images=image, return_tensors="pt")

In [11]:
dummy_input = inputs["pixel_values"]

In [12]:
torch.onnx.export(
    model,
    dummy_input,
    "food_model.onnx",
    input_names=["pixel_values"],
    output_names=["logits"],
    dynamic_axes={"pixel_values": {0: "batch_size"}, "logits": {0: "batch_size"}},
    opset_version=14
)


/home/gitpod/.pyenv/versions/3.12.9/lib/python3.12/site-packages/transformers/models/vit/modeling_vit.py:172: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
/home/gitpod/.pyenv/versions/3.12.9/lib/python3.12/site-packages/transformers/models/vit/modeling_vit.py:178: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if height != self.image_size[0] or width != self.image_size[1]:


In [13]:
import onnx

onnx_model = onnx.load("food_model.onnx")
onnx.checker.check_model(onnx_model)

print("ONNX model is valid!")


ONNX model is valid!


In [14]:
import onnxruntime as ort
import numpy as np

In [15]:
import os 

In [16]:
os.listdir()

['__init__.py',
 'main.py',
 'requirements.txt',
 '.gitignore',
 'env',
 'requirements.dev.txt',
 'playground.ipynb',
 'food_model.onnx']

In [17]:

def get_file_size_in_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)  # Convert bytes to megabytes
    return size_mb

In [18]:
file_path = 'food_model.onnx'

In [19]:
file_size_mb = get_file_size_in_mb(file_path)

In [20]:
file_size_mb

327.8763589859009

In [21]:
from PIL import Image
import numpy as np
import io

# Preprocessing parameters from your config
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

RESCALE_FACTOR = 0.00392156862745098  # Equivalent to dividing by 255
IMAGE_MEAN = [0.5, 0.5, 0.5]
IMAGE_STD = [0.5, 0.5, 0.5]

def preprocess_image(image_bytes: bytes) -> np.ndarray:
    """
    Preprocess an image for inference with ONNX model.

    Args:
        image_bytes (bytes): Raw image bytes

    Returns:
        np.ndarray: Preprocessed image tensor in NCHW format
    """
    # Load the image from bytes and convert to RGB
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    # Resize to 224x224
    image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT), resample=Image.BILINEAR)

    # Convert image to NumPy array (HWC format)
    img_array = np.array(image).astype(np.float32)

    # Rescale pixel values by rescale factor
    img_array *= RESCALE_FACTOR  # Same as dividing by 255

    # Normalize: subtract mean and divide by std
    img_array = (img_array - IMAGE_MEAN) / IMAGE_STD

    # Transpose to CHW format (channels, height, width)
    img_array = np.transpose(img_array, (2, 0, 1))

    # Add batch dimension (1, channels, height, width)
    img_array = np.expand_dims(img_array, axis=0)

    return img_array


In [22]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [23]:
config = model.config

In [26]:
labels = config.label2id

In [29]:
labels

{'apple_pie': '0',
 'baby_back_ribs': '1',
 'baklava': '2',
 'beef_carpaccio': '3',
 'beef_tartare': '4',
 'beet_salad': '5',
 'beignets': '6',
 'bibimbap': '7',
 'bread_pudding': '8',
 'breakfast_burrito': '9',
 'bruschetta': '10',
 'caesar_salad': '11',
 'cannoli': '12',
 'caprese_salad': '13',
 'carrot_cake': '14',
 'ceviche': '15',
 'cheese_plate': '17',
 'cheesecake': '16',
 'chicken_curry': '18',
 'chicken_quesadilla': '19',
 'chicken_wings': '20',
 'chocolate_cake': '21',
 'chocolate_mousse': '22',
 'churros': '23',
 'clam_chowder': '24',
 'club_sandwich': '25',
 'crab_cakes': '26',
 'creme_brulee': '27',
 'croque_madame': '28',
 'cup_cakes': '29',
 'deviled_eggs': '30',
 'donuts': '31',
 'dumplings': '32',
 'edamame': '33',
 'eggs_benedict': '34',
 'escargots': '35',
 'falafel': '36',
 'filet_mignon': '37',
 'fish_and_chips': '38',
 'foie_gras': '39',
 'french_fries': '40',
 'french_onion_soup': '41',
 'french_toast': '42',
 'fried_calamari': '43',
 'fried_rice': '44',
 'frozen

In [30]:
import pickle
with open('label.pkl', 'wb') as file:  # 'wb' = write binary
    pickle.dump(labels, file)

print("Object saved successfully!")


Object saved successfully!


In [31]:
import pickle
with open('label.pkl', 'rb') as file:
    loaded_labels = pickle.load(file)
print("Object loaded successfully!")
print(loaded_labels)


Object loaded successfully!
{'apple_pie': '0', 'baby_back_ribs': '1', 'baklava': '2', 'beef_carpaccio': '3', 'beef_tartare': '4', 'beet_salad': '5', 'beignets': '6', 'bibimbap': '7', 'bread_pudding': '8', 'breakfast_burrito': '9', 'bruschetta': '10', 'caesar_salad': '11', 'cannoli': '12', 'caprese_salad': '13', 'carrot_cake': '14', 'ceviche': '15', 'cheese_plate': '17', 'cheesecake': '16', 'chicken_curry': '18', 'chicken_quesadilla': '19', 'chicken_wings': '20', 'chocolate_cake': '21', 'chocolate_mousse': '22', 'churros': '23', 'clam_chowder': '24', 'club_sandwich': '25', 'crab_cakes': '26', 'creme_brulee': '27', 'croque_madame': '28', 'cup_cakes': '29', 'deviled_eggs': '30', 'donuts': '31', 'dumplings': '32', 'edamame': '33', 'eggs_benedict': '34', 'escargots': '35', 'falafel': '36', 'filet_mignon': '37', 'fish_and_chips': '38', 'foie_gras': '39', 'french_fries': '40', 'french_onion_soup': '41', 'french_toast': '42', 'fried_calamari': '43', 'fried_rice': '44', 'frozen_yogurt': '45', '